### Modelo de Flujo de Caja (Cash Flow)

Este notebook contiene dos modelos de analisis financiero que tienen como objeto de estudio los diferentes modelo de VE que se encuentran en el Dataset "ElectricCarData_Norm.csv". Estos modelos estudian el comportamiento financiero de cada prototipo del vehiculo materializado en el calculo de un Flujo de Caja (Cash Flow) proyectado a 5 años, e indicadores financiero como: ROI (Return on Investment), IR (Indice de Rentabilidad) y Payback Period.

El primer modelo recibe como parametro una instancia (modelo de VE) del dataset y devuelve como resultado el comportamiento de este en relacion a las variables financieras anteriormente expuestas y tambien el del un auto convencional para obtener un analisis comparativo y poder evaluar diferencias y el costo de oportunidad de estas dos variantes.


Importamos las librerias necesarias.

In [ ]:
import pandas as pd 


Importamos el dataset que contiene las instancias y variables necesarias. (Este dataset contiene los valores futuros, producto del modelo de ML)

In [12]:
future_data = pd.read_csv(r"Prediccion_Dataset.csv", index_col = 0)

Visualizamos el Dataset.

In [13]:
future_data.head(5)

,Income_per_Vehicle (USD),Miles_per_Vehicle,Hours_Per_Driver
Year,,,
2025,65357.06,11762.56,1939.06
2026,64802.76,11703.94,1643.53
2027,64302.32,11656.52,1348.03
2028,63485.52,11556.92,1050.00
2029,63408.43,11587.08,756.66


Utilizamos como instrumento de ponderacion la media o promedio para tener un primer acercamiento y entendimiento a las variables (ingreso anual y millas recorridas al año). 

In [14]:
promedio_ingresos = round(future_data["Income_per_Vehicle (USD)"].mean(),2)
promedio_millas = round(future_data["Miles_per_Vehicle"].mean(),2)
print(f'El promedio de ingresos anuales es: {promedio_ingresos}')
print(f'El promedio de millas recorridas anualmente es : {promedio_millas}')

El promedio de ingresos anuales es: 64271.22
El promedio de millas recorridas anualmente es : 11653.4


Se asigna una variable a cada dato.

In [15]:
año = 2025
for _, row in future_data.iterrows():  # iterrows() en lugar de interrows()
    globals()[f'Ingresos_año_{año}'] = row["Income_per_Vehicle (USD)"]
    print(f'Los ingresos para el año {año} son {globals()[f"Ingresos_año_{año}"]}')
    año += 1




Los ingresos para el año 2025 son 65357.06
Los ingresos para el año 2026 son 64802.76
Los ingresos para el año 2027 son 64302.32
Los ingresos para el año 2028 son 63485.52
Los ingresos para el año 2029 son 63408.43


In [16]:
ingresos_años = []

for _, row in future_data.iterrows():
    ingresos_años.append(row["Income_per_Vehicle (USD)"])
print(ingresos_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43]


In [17]:
año = 2025

for _, row in future_data.iterrows():
    globals()[f'Millas_Año_{año}'] = row["Miles_per_Vehicle"]
    print(f'Las Millas recorridas en el año {año} son {globals()[f"Millas_Año_{año}"]}')
    año+=1

Las Millas recorridas en el año 2025 son 11762.56
Las Millas recorridas en el año 2026 son 11703.94
Las Millas recorridas en el año 2027 son 11656.52
Las Millas recorridas en el año 2028 son 11556.92
Las Millas recorridas en el año 2029 son 11587.08


In [18]:
millas_años = []
for _, row in future_data.iterrows():
    millas_años.append(row["Miles_per_Vehicle"])
print(millas_años)

[11762.56, 11703.94, 11656.52, 11556.92, 11587.08]


In [19]:
print(ingresos_años)
print(millas_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43]
[11762.56, 11703.94, 11656.52, 11556.92, 11587.08]


En esta sección se elabora el mecanismo para el cálculo del modelo de Flujo de Caja (Cash Flow) y los demas indicadores financieros.

A continuación se presentaran las siguientes características:

- Debido a que son dos tipos de autos con caracteristicas distintas se crean 2 clases para cada variante, que heredan de la clase madre «Auto».
- Las caracteristicas que estructuran la clase «Auto Convencional» corresponden a una ponderacion que representa un propotipo de auto convencional promedio
- Se crean funciones para el calculo de las variables financieras anteriormente mencionadas.
- El objeto de análisis es un modelo de VE que compone el dataset y tambien de un prototipo promedio de un auto convencional.
- El resultado es el dictámen financiero compuesto por las variables financieras expuestas.
- Obtención de análisis comparativo entre un VE y un auto convencional.
- Análisis Financiero de la totalidad de VE y ranking de los prototipos más rentables.

### Modelo de Flujo de Caja (Cash Flow) #1
Análisis Financiero personalizado (por modelo VE)

In [ ]:
# Dataset de VE
df_autos = pd.read_csv("df_modelo.csv")

# Se crea la clase Auto y subclases para AutoConvencional y AutoElectrico
class Auto:
    def __init__(self, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento):
        self.precio = precio
        self.iva = iva
        self.licencia_1 = licencia_1
        self.licencia_2 = licencia_2
        self.placa = placa
        self.inspeccion = inspeccion
        self.seguro = seguro
        self.recorrido_anual = recorrido_anual
        self.ingresos_anuales = ingresos_anuales
        self.tasa_descuento = tasa_descuento
    
    def inversion_inicial(self):
        raise NotImplementedError
    
    def costos_operativos(self):
        raise NotImplementedError
    
    def flujo_neto_anual(self):
        return self.ingresos_anuales - self.costos_operativos()
    
    def flujo_caja_proyectado(self, años=5):
        # Inversión inicial negativa porqué es el año 0
        inversion_inicial = -self.inversion_inicial()
        
        ingresos_año = [0]
        costos_año = [0]
        flujo_neto = [0]  # Año 0: 
        flujo_descuento = [inversion_inicial]  # Año 0 Inversión inicial negativa
        # Se calcula el flujo neto anual y mensual para cada año
        for año in range(1, años + 1):
            self.ingresos_anuales = ingresos_años[año-1] # Codigo nuevo
            self.recorrido_anual = millas_años[año-1] # Codigo nuevo
            flujo_neto_actual = self.flujo_neto_anual()
            flujo_neto.append(flujo_neto_actual)
            flujo_descuento_actual = flujo_neto_actual / (1 + self.tasa_descuento) ** año
            flujo_descuento.append(flujo_descuento_actual)
            ingresos_año.append(self.ingresos_anuales)
            costos_año.append(self.costos_operativos())
        
        # Se crea el DataFrame de resultados
        df_flujo_caja = pd.DataFrame({
            'Año': range(0, años + 1),
            'Ingresos': ingresos_año, 
            'Costos Operativos': costos_año,
            'Flujo Neto': flujo_neto,
            'Flujo Neto Descontado (10%)': flujo_descuento
        })
         # Calculo el flujo total descontado para el ROI
        flujo_total = sum(flujo_descuento)
        return df_flujo_caja

    def calcular_roi(self, flujo_total):
        roi = (flujo_total - self.inversion_inicial()) / self.inversion_inicial()
        return round(roi, 2)

    def calcular_ir(self, flujo_total):
        ir = flujo_total / abs(self.inversion_inicial())
        return round(ir, 2)

    def calcular_payback_period(self):
        flujo_acumulado = 0
        payback = 0
        for año in range(1, 11):
            flujo_anual = self.flujo_neto_anual()
            flujo_acumulado += flujo_anual
            if flujo_acumulado >= abs(self.inversion_inicial()):
                payback = año
                break
        return payback if payback > 0 else "No recuperado en 10 años"

class AutoConvencional(Auto): #Licencia_1 = unico pago, licencia_2 = anual
    def __init__(self, precio=38000, iva=0.08875, licencia_1=550,licencia_2=100, placa = 300,
                 seguro=2500, inspeccion = 150,
                 eficiencia=0.06, mantenimiento=0.15, recorrido_anual=promedio_millas, #mantenimiento = USD x Milla
                 ingresos_anuales=promedio_ingresos, tasa_descuento=0.10):
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion,
                         seguro, recorrido_anual, ingresos_anuales, tasa_descuento) # Se usa el super constructor para evitar redundancia ya que es una clase hija
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
    
    def inversion_inicial(self):
        return self.precio * (1 + self.iva) + self.licencia_1
    
    def costos_operativos(self):
        precio_gasolina_litro = 0.92 #USD x lt
        costo_gasolina_anual = (self.recorrido_anual * self.eficiencia) * precio_gasolina_litro
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        return costo_gasolina_anual + costo_mantenimiento + self.seguro + self.licencia_2 + self.placa + self.inspeccion

class AutoElectrico(Auto): #Descuento de 5% al 10% en licencia ; Excencion del impuesto de venta
    def __init__(self, precio, eficiencia, iva=0, licencia_1=495,licencia_2=100, placa = 300,
                 seguro=1500, inspeccion = 100, mantenimiento=0.03, wallbox=600, 
                 recambio_bateria=5000, #mantenimiento = USD x Milla
                 recorrido_anual=16529, ingresos_anuales=93227, tasa_descuento=0.10):
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento)
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
        self.wallbox = wallbox
        self.recambio_bateria = recambio_bateria
    
    def inversion_inicial(self):
        return self.precio + self.licencia_1 + self.wallbox
    
    def costos_operativos(self):
        precio_electricidad_kWh = 0.13 #USD x kHw
        costo_electricidad_anual = (self.recorrido_anual * self.eficiencia) * precio_electricidad_kWh
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_bateria = self.recambio_bateria / 12.5
        return costo_electricidad_anual + costo_mantenimiento + costo_anual_bateria + self.seguro + self.licencia_2 + self.placa + self.inspeccion

# Función para seleccionar el modelo del auto, calcular flujos e indicadores
def comparar_flujo_caja_y_medir_indicadores(modelo, tipo_auto):
    # Se Filtra en el dataset para obtener el precio del modelo
    filtro_modelo = df_autos[df_autos['Model'] == modelo]
    
    if filtro_modelo.empty:
        print(f"Modelo '{modelo}' no encontrado en el dataset.")
        return None
    
    precio = filtro_modelo['Precio Dolar'].values[0]
    eficiencia = filtro_modelo['Efficiency (kWh/mile)'].values[0]
    
    # Instancio el auto eléctrico y el auto convencional
    auto_electrico = AutoElectrico(precio, eficiencia)
    auto_convencional = AutoConvencional()
    
    # Se calcula el flujo de caja proyectado
    flujo_electrico = auto_electrico.flujo_caja_proyectado()
    flujo_convencional = auto_convencional.flujo_caja_proyectado()
    
    # Sumo flujo neto descontado para cada tipo de auto (obtener total)
    flujo_descuento_total_electrico = flujo_electrico['Flujo Neto Descontado (10%)'].sum()
    flujo_descuento_total_convencional = flujo_convencional['Flujo Neto Descontado (10%)'].sum()
    
    # Calculo del ROI, IR y Payback Period para cada auto
    roi_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
    roi_convencional = auto_convencional.calcular_roi(flujo_descuento_total_convencional)
    ir_electrico = auto_electrico.calcular_ir(flujo_descuento_total_electrico)
    ir_convencional = auto_convencional.calcular_ir(flujo_descuento_total_convencional)
    payback_electrico = auto_electrico.calcular_payback_period()
    payback_convencional = auto_convencional.calcular_payback_period()
    
    df_flujo_electrico = pd.DataFrame(flujo_electrico)
    df_flujo_convencional = pd.DataFrame(flujo_convencional)

    # Resultados
    print(f"Flujo de Caja Proyectado para {modelo} (Eléctrico):")
    print(df_flujo_electrico)
    
    print("\nFlujo de Caja Proyectado para Auto Convencional:")
    print(df_flujo_convencional)
    #df_flujo_convencional
    print("\nComparación Financiera:")
    print(f"Auto Eléctrico - Flujo Neto Descontado Total: {flujo_descuento_total_electrico}, ROI: {roi_electrico}%, IR: {ir_electrico} USD, Payback Period: {payback_electrico} años")
    print(f"Auto Convencional - Flujo Neto Descontado Total: {flujo_descuento_total_convencional}, ROI: {roi_convencional}%, IR: {ir_convencional} USD, Payback Period: {payback_convencional} años")




Ejemplo de uso:

In [ ]:
modelo_auto =  'e-Niro 39 kWh'  # El modelo elegido
comparar_flujo_caja_y_medir_indicadores(modelo_auto, AutoElectrico)

Flujo de Caja Proyectado para e-Niro 39 kWh (Eléctrico):
   Año  Ingresos  Costos Operativos    Flujo Neto  Flujo Neto Descontado (10%)
0    0      0.00           0.000000      0.000000                -37559.000000
1    1  65357.06        2911.906611  62445.153389                 56768.321263
2    2  64802.76        2909.355469  61893.404531                 51151.573993
3    3  64302.32        2907.291750  61395.028250                 46126.993426
4    4  63485.52        2902.957158  60582.562842                 41378.705581
5    5  63408.43        2904.269722  60504.160278                 37568.323251

Flujo de Caja Proyectado para Auto Convencional:
   Año  Ingresos  Costos Operativos    Flujo Neto  Flujo Neto Descontado (10%)
0    0      0.00           0.000000      0.000000                -41922.500000
1    1  65357.06        5463.677312  59893.382688                 54448.529716
2    2  64802.76        5451.648488  59351.111512                 49050.505382
3    3  64302.32        

A continuacion se exponen los modelos contenidos en el dataset para agregarlos en le input de la funcion.

In [22]:
df_autos["Model"].unique()

array(['Model 3 Long Range Dual Motor', 'ID.3 Pure', '2', 'iX3 ', 'e ',
       'Air ', 'e-Golf ', 'e-208 ', 'Model 3 Standard Range Plus',
       'Q4 e-tron ', 'EQC 400 4MATIC', 'Leaf ', 'Kona Electric 64 kWh',
       'i4 ', 'IONIQ Electric', 'ID.3 Pro S', 'Taycan Turbo S', 'e-Up! ',
       'ZS EV', 'Cooper SE ', 'Corsa-e ', 'Model Y Long Range Dual Motor',
       'Enyaq iV 50', 'e-tron GT ', 'Model 3 Long Range Performance',
       'ID.4 ', 'ID.3 Pro', 'XC40 P8 AWD Recharge', 'i3 120 Ah',
       'e-2008 SUV ', 'e-tron 50 quattro', 'e-Niro 64 kWh',
       'Zoe ZE50 R110', 'Cybertruck Tri Motor', 'MX-30 ', 'Leaf e+',
       'UX 300e', 'el-Born ', 'Zoe ZE50 R135', 'EQA ',
       'Model S Long Range', 'Kona Electric 39 kWh',
       'e-tron Sportback 55 quattro', 'CITIGOe iV ', 'Mii Electric ',
       'e-Soul 64 kWh', 'Ampera-e ', 'Taycan 4S', 'One ', 'U5 ',
       'e-tron 55 quattro', 'Roadster ', 'Mokka-e ', 'Enyaq iV 80',
       'Model X Long Range', 'e Advance', '3 Crossback E-Tense',


### Modelo de Flujo de Caja (Cash Flow) #2:
Ranking de rentabilidad financiera de todos los prototipos de VE contenidos en el dataset

In [ ]:
# Definición de la función para calcular y comparar métricas
def calcular_metricas_para_todos_los_modelos(df_autos):
    resultados = []
    resultados_conv = []

    # Se itera sobre cada modelo en el dataset
    for _, row in df_autos.iterrows(): #uso _, ya que no necesito el indice
        modelo = row['Model']
        precio = row['Precio Dolar']
        eficiencia = row['Efficiency (kWh/mile)']
        # Se instancia AutoEléctrico y AutoConvencional con el precio del modelo actual
        auto_electrico = AutoElectrico(precio, eficiencia)
        auto_convencional = AutoConvencional()

        # Se calcula los flujos de caja proyectados para ambos tipos de autos
        flujo_electrico = auto_electrico.flujo_caja_proyectado()
        flujo_convencional = auto_convencional.flujo_caja_proyectado()

        # Luego, se calcula el flujo neto descontado total
        flujo_descuento_total_electrico = flujo_electrico['Flujo Neto Descontado (10%)'].sum()
        flujo_descuento_total_convencional = flujo_convencional['Flujo Neto Descontado (10%)'].sum()

        # Calculo ROI, IR y Payback Period
        roi_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
        ir_electrico = auto_electrico.calcular_ir(flujo_descuento_total_electrico)
        payback_electrico = auto_electrico.calcular_payback_period()

        # Se agg los resultados a la lista
        resultados.append({
            'Modelo': modelo,
            'Precio (USD)': round(precio,1),
            'Flujo Neto Descontado (USD)': round(flujo_descuento_total_electrico,1),
            'ROI (%)': roi_electrico,
            'IR (USD)': ir_electrico,
            'Payback Period (Años)': payback_electrico
        })

        # Se convierte a DataFrame
    df_resultados = pd.DataFrame(resultados)
    #df_resultados_conv = pd.DataFrame(resultados_conv)
    # Se obtienen los 5 mejores con nlargest
    #top_5_modelos = df_resultados.nlargest(5, ['ROI (%)', 'IR (USD)', 'Flujo Neto Descontado (USD)'])
    df_resultados = df_resultados.sort_values(by="ROI (%)", ascending=False)

    return df_resultados
    #return top_5_modelos

# Se invoca la función para calcular métricas y obtener los top 5 modelos
df_resultados = calcular_metricas_para_todos_los_modelos(df_autos)

df_resultados.head(5)

,Modelo,Precio (USD),Flujo Neto Descontado (USD),ROI (%),IR (USD),Payback Period (Años)
44,Mii Electric,21336.7,210567.9,8.39,9.39,1
82,EQ fortwo coupe,22670.2,209228.7,7.80,8.80,1
17,e-Up!,22706.3,209198.4,7.79,8.79,1
77,EQ forfour,23351.8,208518.4,7.53,8.53,1
43,CITIGOe iV,26006.0,205898.6,6.60,7.60,1


Por último se exportan los resultados a un archivo .csv para crear un nuevo Dataset que contenga información financiera de cada prototipo de VE.

In [ ]:
df_resultados.to_csv(r"C:\Users\Cristian\Desktop\DATA SCIENCE\HENRY\P_Final\Codigo\Cash_Flow\Analisis_Financiero_VE.csv", index = False)